# Path
1. Create the model in GCP
    - Create the Dialogflow structure
    - Implement the PaLM into the Dialogflow
    - Test it out 
2. Make the backend locally
3. Create a Front End
4. Deploy the Front End

## Future Improvements
- Add identification in the frontend and change the conversational model if the user is identified. For example in the welcome intent instead of saying the fir interaction, get it ready to do something

---

# Imports
Importing all the libraries needed to run the notebook

In [23]:
from google.cloud import dialogflow
import uuid
from google.oauth2 import service_account
import os
from dotenv import load_dotenv
import gradio as gr
import random
import time

In [24]:
# Load the environment variables
load_dotenv()

True

# Agent creation
---
**ADD IMAGES ONCE FINISHED**


# Agent connection
---
In order to connect to the project created in GCP and tHe agent created in Dialogflow we will have to follow a series of steps starting from authentication
### Authentication
Authentication as a service account in the Google Cloud Platform to access the resources in the cloud

In [25]:
# Authenticate the computer as a service account
service_account_key = os.getenv('service_account_key')
credentials = service_account.Credentials.from_service_account_file(service_account_key)

### Session Creation
A session represents a conversation between a Dialogflow agent and an end-user. You create a session at the beginning of a conversation and use it for each turn of the conversation. Once the conversation has ended, you discontinue using the session

In [26]:
session_id = uuid.uuid4().hex

In [27]:
# Load the variables to access the databases
project_id = os.getenv('project_id')
language_code = 'en'

In [28]:
# Create the session with the appropriate credentials
session_client = dialogflow.SessionsClient(credentials=credentials)

session = session_client.session_path(project_id, session_id)

### Agent function
Function that given an input returns the output of by connecting to the agent

In [29]:
def dialogflow_request(text, language_code):
    text_input = dialogflow.TextInput(text=text, language_code=language_code)
    query_input = dialogflow.QueryInput(text=text_input)
    response = session_client.detect_intent(
        request={"session": session, "query_input": query_input}
    )
    return response

# Web Application 
---
Web application using Gradio to interact with the agent

In [30]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def respond(message, chat_history):
        #We add our function
        response = dialogflow_request(message, 'en')
        #We get the answer from the response of Dialogflow
        bot_message = str(response.query_result.fulfillment_messages[0].text.text[0])
        chat_history.append((message, bot_message))
        time.sleep(1)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


# TRYOUT

In [ ]:
# TRYOUT
text="Hello"

text_input = dialogflow.TextInput(text=text, language_code=language_code)
query_input = dialogflow.QueryInput(text=text_input)
query_input

In [ ]:
response = session_client.detect_intent(
        request={"session": session, "query_input": query_input}
    )
response